# Getting started

### CLEF 2025 - CheckThat! Lab  - Task 4 Scientific Web Discourse - Subtask 4b (Scientific Claim Source Retrieval)

This notebook enables participants of subtask 4b to quickly get started. It includes the following:
- Code to upload data, including:
    - code to upload the collection set (CORD-19 academic papers' metadata)
    - code to upload the query set (tweets with implicit references to CORD-19 papers)
- Code to run a baseline retrieval model (BM25)
- Code to evaluate the baseline model

Participants are free to use this notebook and add their own models for the competition.

# 1) Importing data

In [9]:
import numpy as np
import pandas as pd

## 1.a) Import the collection set
The collection set contains metadata of CORD-19 academic papers.

The preprocessed and filtered CORD-19 dataset is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4?ref_type=heads

Participants should first download the file then upload it on the Google Colab session with the following steps.


In [53]:
# 1) Download the collection set from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4?ref_type=heads
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_COLLECTION_DATA = 'subtask4b_collection_data.pkl' #MODIFY PATH

In [54]:
df_collection = pd.read_pickle(PATH_COLLECTION_DATA)

In [12]:
df_collection.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7718 entries, 162 to 1056448
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   cord_uid          7718 non-null   object        
 1   source_x          7718 non-null   object        
 2   title             7718 non-null   object        
 3   doi               7677 non-null   object        
 4   pmcid             4959 non-null   object        
 5   pubmed_id         6233 non-null   object        
 6   license           7718 non-null   object        
 7   abstract          7718 non-null   object        
 8   publish_time      7715 non-null   object        
 9   authors           7674 non-null   object        
 10  journal           6668 non-null   object        
 11  mag_id            0 non-null      float64       
 12  who_covidence_id  528 non-null    object        
 13  arxiv_id          20 non-null     object        
 14  label             7718 n

In [ ]:
df_collection.head()

,cord_uid,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,label,time,timet
162,umvrwgaw,PMC,Professional and Home-Made Face Masks Reduce E...,10.1371/journal.pone.0002618,PMC2440799,18612429,cc-by,BACKGROUND: Governments are preparing for a po...,2008-07-09,"van der Sande, Marianne; Teunis, Peter; Sabel,...",PLoS One,NaN,NaN,NaN,umvrwgaw,2008-07-09,1215561600
611,spiud6ok,PMC,The Failure of R (0),10.1155/2011/527610,PMC3157160,21860658,cc-by,"The basic reproductive ratio, R (0), is one of...",2011-08-16,"Li, Jing; Blakeley, Daniel; Smith?, Robert J.",Comput Math Methods Med,NaN,NaN,NaN,spiud6ok,2011-08-16,1313452800
918,aclzp3iy,PMC,Pulmonary sequelae in a patient recovered from...,10.4103/0970-2113.99118,PMC3424870,22919170,cc-by-nc-sa,The pandemic of swine flu (H1N1) influenza spr...,2012,"Singh, Virendra; Sharma, Bharat Bhushan; Patel...",Lung India,NaN,NaN,NaN,aclzp3iy,2012-01-01,1325376000
993,ycxyn2a2,PMC,What was the primary mode of smallpox transmis...,10.3389/fcimb.2012.00150,PMC3509329,23226686,cc-by,The mode of infection transmission has profoun...,2012-11-29,"Milton, Donald K.",Front Cell Infect Microbiol,NaN,NaN,NaN,ycxyn2a2,2012-11-29,1354147200
1053,zxe95qy9,PMC,"Lessons from the History of Quarantine, from P...",10.3201/eid1902.120312,PMC3559034,23343512,no-cc,"In the new millennium, the centuries-old strat...",2013-02-03,"Tognotti, Eugenia",Emerg Infect Dis,NaN,NaN,NaN,zxe95qy9,2013-02-03,1359849600


## 1.b) Import the query set

The query set contains tweets with implicit references to academic papers from the collection set.

The preprocessed query set is available on the Gitlab repository here: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4?ref_type=heads

Participants should first download the file then upload it on the Google Colab session with the following steps.

In [55]:
# 1) Download the query tweets from the Gitlab repository: https://gitlab.com/checkthat_lab/clef2025-checkthat-lab/-/tree/main/task4?ref_type=heads
# 2) Drag and drop the downloaded file to the "Files" section (left vertical menu on Colab)
# 3) Modify the path to your local file path
PATH_QUERY_DATA = 'subtask4b_query_tweets_dev.tsv' #MODIFY PATH

In [56]:
df_query = pd.read_csv(PATH_QUERY_DATA, sep = '\t')

In [ ]:
df_query.head()

,tweet_text,cord_uid
0,Oral care in rehabilitation medicine: oral vul...,htlvpvz5
1,this study isn't receiving sufficient attentio...,4kfl29ul
2,"thanks, xi jinping. a reminder that this study...",jtwb17u8
3,Taiwan - a population of 23 million has had ju...,0w9k8iy1
4,Obtaining a diagnosis of autism in lower incom...,tiqksd69


In [51]:
df_query.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   post_id     1400 non-null   int64 
 1   tweet_text  1400 non-null   object
 2   cord_uid    1400 non-null   object
dtypes: int64(1), object(2)
memory usage: 32.9+ KB


In [57]:
np.random.seed(88)
df_query = df_query.sample(20)

# 2) Running the baseline
The following code runs a BM25 baseline.


In [58]:
!pip install rank_bm25
from rank_bm25 import BM25Okapi

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
distutils: /storage/home/hcoda1/3/stian40/.local/lib/python3.9/site-packages
sysconfig: /storage/home/hcoda1/3/stian40/.local/lib64/python3.9/site-packages
user = True
home = None
root = None
prefix = None


In [59]:
# Create the BM25 corpus
corpus = df_collection[:][['title', 'abstract']].apply(lambda x: f"{x['title']} {x['abstract']}", axis=1).tolist()
cord_uids = df_collection[:]['cord_uid'].tolist()
tokenized_corpus = [doc.split(' ') for doc in corpus]
bm25 = BM25Okapi(tokenized_corpus)

In [60]:
text2bm25top = {}

def get_top_cord_uids(query):
    if query in text2bm25top.keys():
        return text2bm25top[query]
    else:
        tokenized_query = query.split(' ')
        doc_scores = bm25.get_scores(tokenized_query)
        indices = np.argsort(-doc_scores)[:1000]
        bm25_topk = [cord_uids[x] for x in indices]

        text2bm25top[query] = bm25_topk
        return bm25_topk


In [61]:
# Retrieve topk candidates using the BM25 model
df_query['bm25_topk'] = df_query['tweet_text'].apply(lambda x: get_top_cord_uids(x))

# 3) Evaluating the baseline
The following code evaluates the BM25 retrieval baseline on the query set using the Mean Reciprocal Rank score (MRR@5).

In [62]:
# Evaluate retrieved candidates using MRR@k
def get_performance_mrr(data, col_gold, col_pred, list_k = [1, 5, 10]):
    d_performance = {}
    for k in list_k:
        data["in_topx"] = data.apply(lambda x: (1/([i for i in x[col_pred][:k]].index(x[col_gold]) + 1) if x[col_gold] in [i for i in x[col_pred][:k]] else 0), axis=1)
        #performances.append(data["in_topx"].mean())
        d_performance[k] = data["in_topx"].mean()
    return d_performance


In [63]:
results = get_performance_mrr(df_query, 'cord_uid', 'bm25_topk')
# Printed MRR@k results in the following format: {k: MRR@k}
print(results)

{1: np.float64(0.35), 5: np.float64(0.4125), 10: np.float64(0.4196428571428571)}


# 4) Possible solutions

# 4.1) BM25 for Retrieval and SBert for Rerank

In [1]:
!pip install rerankers
# All of the above
!pip install "rerankers[all]"

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 51 kB 454 kB/s eta 0:00:011
  distutils: /storage/home/hcoda1/3/stian40/.local/lib/python3.9/site-packages
  sysconfig: /storage/home/hcoda1/3/stian40/.local/lib64/python3.9/site-packages
  user = True
  home = None
  root = None
  prefix = None
Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 316 kB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 766.7 MB 7.0 kB/s eta 0:00:01     |███████████████████▉            | 474.6 MB 3.0 MB/s eta 0:01:37��████████████████▋       | 588.8 MB 1.8 MB/s eta 0:01:39     |██████████████████████████▉     | 642.4 MB 1.2 MB/s eta 0:01:47��████████▏    | 649.7 MB 1.2 MB/s eta 0:01:41
     |████████████████████████████████| 7.6 MB 84.0 MB/s eta 0:00:01
     |████████████████████████████████| 1.3 MB 100.9 MB/s eta 0:00:01
     |████████████████████████████████| 

In [64]:
from rerankers import Reranker, Document

In [65]:
# default “cross‑encoder” is ms‑marco‑MiniLM‑L‑6‑v2 under the hood
reranker = Reranker("cross-encoder/ms-marco-MiniLM-L-6-v2", model_type="cross-encoder")
# — or explicitly:
# reranker = Reranker("cross‑encoder/ms‑marco‑MiniLM‑L‑6‑v2", model_type="cross‑encoder")

Loading TransformerRanker model cross-encoder/ms-marco-MiniLM-L-6-v2
No device set
Using device cpu
No dtype set
Device set to `cpu`, setting dtype to `float32`
Using dtype torch.float32
Loaded model cross-encoder/ms-marco-MiniLM-L-6-v2
Using device cpu.
Using dtype torch.float32.


In [66]:
def retrieve_paper(paper_ids):
    paper_dict = {}
    for pid in paper_ids:
        paper_data = df_collection[df_collection['cord_uid'] == pid]
        if not paper_data.empty:
            title = paper_data.iloc[0]['title']
            abstract = paper_data.iloc[0]['abstract']
            paper_dict[pid] = {'title': title, 'abstract': abstract}
    return paper_dict

df_query['title_abstract'] = df_query['bm25_topk'].apply(retrieve_paper)


In [67]:
df_query[['tweet_text', 'title_abstract']].head()

,tweet_text,title_abstract
359,wow. i said in 2020 that bats are in a mutual...,{'po6cqv4m': {'title': 'Seroreactive profiling...
1151,"so remdesivir, some positive news, some negati...",{'gckr8h1m': {'title': 'The effect of SARS-CoV...
82,"in this research, we noted significant socioec...",{'bcttenl4': {'title': 'Excess years of life l...
360,"According to this Danish study, after 90 days,...",{'dptapovm': {'title': 'Infection and vaccine-...
343,we modeled contact tracing strategies (with )...,{'vdejfuuy': {'title': 'Impact of delays on ef...


In [68]:
def rerank_with_rerankers(row, top_k=10):
    query = row["tweet_text"]

    # Ensure input format for Reranker
    docs = [
        f"{info['title']} {info['abstract']}"
        for info in row["title_abstract"].values()
    ]
    doc_ids = list(row["title_abstract"].keys())

    # Apply reranker
    results = reranker.rank(query=query, docs=docs, doc_ids=doc_ids)

    # Get top_k reordered cord_uids
    top_results = results.top_k(top_k)
    return [res.doc_id for res in top_results]

# Apply to DataFrame
df_query["bm25_reranker_topk"] = df_query.apply(rerank_with_rerankers, axis=1)

KeyboardInterrupt: 

In [ ]:
mrr_scores = get_performance_mrr(
    df_query,                # your DataFrame
    "cord_uid",              # ground‑truth column
    "bm25_reranker_topk"     # our new reranked lists
)
print(mrr_scores)

In [ ]:
def rerank_with_crossencoder(row):
    tweet = row['tweet_text']
    title_abstracts = row['title_abstract']

    model_inputs = [(tweet, f"{paper_data['title']} {paper_data['abstract']}")
                    for paper_id, paper_data in title_abstracts.items()]

    results = cross_encoder.rank(query=tweet, documents=[f"{paper_data['title']} {paper_data['abstract']}"
                                                 for paper_id, paper_data in title_abstracts.items()],
                         top_k=10, show_progress_bar = True)

    ranked_document_indices = [result['corpus_id'] for result in results]

    ranked_paper_ids = [list(title_abstracts.keys())[index] for index in ranked_document_indices]

    return ranked_paper_ids

df_query['bm25_cross_encoder_topk'] = df_query.apply(lambda row: rerank_with_crossencoder(row), axis=1)

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

KeyError: "['bm25_cross_encoder_topk'] not in index"

In [ ]:
# Check the result (this will contain the tweet and paper pairs)
df_query[['tweet_text', 'cord_uid', 'bm25_topk', 'bm25_cross_encoder_topk']].head()

,tweet_text,cord_uid,bm25_topk,cross_encoder_topk
8073,Sinovac. new safety/immunogenicity results in a participant's aged >18 yrs in phase 3 trial in Chile. Interim report: Safety/immunogenicity of an inactivated vaccine against SARS-CoV-2 in healthy Chilean adults in a phase 3 clinical trial | medRxiv,rcb943w2,"[rcb943w2, ssdqobqb, amw2c96l, dkfohub3, so6irh9b, rda2kmv1, kff7ho04, qa1tbu6t, 1uabldiq, 0pptqw7o, i1icueuw, r7aqsyfl, 6akx6xpt, awfgfmx1, ehmy9al1, tl39u0h9, 4vuucmfd, wdtrpnkn, hmkqnply, y77rbrnd, k2jzadwq, ukbhlaaa, 86qczmbt, nk8m2dxt, 3ncm8iwb, vxf2jexb, kdxoyvsv, tq8jpmin, hay91kuq, ztwemu74, uj5deryi, r4fd5vp6, t90pvtad, 9uxazk87, zbi0llig, nvtoxzka, 6274gicp, 4zcjjoc7, g2wpw330, 0n07fow0, 5rkxs2w3, z8758x5u, wabd3b9z, gh6bizzh, 5gshj480, zql4zhyk, 3g8asqkb, ydh4ve24, v7szuzfa, o71p89nb, y40s45iv, pvw9t7zd, z9jqbliw, ifxm3j4y, tz8gc3cz, 3qefts96, 7wsumadw, 3jfok14h, 8hbll61z, 5ojyly05, s86zb5up, 8x3lqokw, vo4csaah, 8ljnxihr, lyhdeks1, 8pz6131o, z218oii3, wt6azxc1, xj1nw76b, mck3rgcm, ix48o00j, ro29tdt8, 1hlqov6m, rpa2v44v, 7z6e5jhh, 4ttda53p, s4xintwp, u7tqolyi, bi9fid5f, 0igh53pl, h4sbz3md, mldwnz43, lq580iwg, 7twi1z5b, a45f2qtl, 1gv3t5t0, olv2kuwx, fdkwklzj, m1bvurwi, e0pz0z4j, mqyu72nc, rng65ofx, c1db9jlv, uexahhdr, 7iuls8x1, 8t2tic9n, 9jlnker4, wdfzrzkt, 5vp2r2bd, fvcyqzgb, ...]","[rcb943w2, 1uabldiq, dkfohub3, 0pptqw7o, ssdqobqb, amw2c96l, kdxoyvsv, rda2kmv1, hmkqnply, nvtoxzka]"
4754,"before discussing #vaccine impacts, is there a link between the ongoing #covid-19 pandemic &amp; rates of #erectiledysfunction❓ 🎉here is our #systematic review that examines this topic, taking into account both exposure and #lockdown effects.",lcqxrcji,"[8rv94jeu, l69kwh48, zmkvs7e7, j0uojajt, e2rtzymt, sxx3yid9, mkl7rvr6, wu2dyleo, 44hifagu, f0m3wuyj, o5zkv42f, cpbu3fv3, a0q61mpi, uppeztta, dbxufi3m, 4ohgr8j7, 6gcgzx36, g9ao6ruo, 2i7zd1t2, sijfzd2o, kt47i60w, l6zsf85e, l375v2my, 57qk34fx, fer4tlhz, ws7lp560, 3a7utmsd, 0t5n87u6, bk3ss63e, b0g8y96j, 4bbapd1t, 0lk8eujq, aq8ckfv7, cymppa7g, styavbvi, umyojne2, hsm75cww, v7udqoi0, o9xdv4x0, 42wcyjuk, p9qeaedh, 8hvve871, r5552ql7, b9b6ww9q, ekal5251, nemkary2, 9gnqfmbq, azr73w2w, z65wm00n, xdbtwiqw, otjnwb4s, wfa5fb3r, ib17jyva, lcqxrcji, ouno4jpl, n119a7fx, 5wlbalzu, oud5ioks, xxy10tcl, gg5c8v7d, 60e9lhfi, sh9w4ipq, c80hle1p, sajmj1jw, r4j1rcm5, 6sy80720, 2rx84imv, ierqfgo5, yslf5er3, 0ho4f5wg, 3lzm42wq, yq6jhupe, ueh60bz2, gatxuwz7, 8w2t6u96, scw4p9wq, mfop9fne, 3ees74tt, 1m2mp7o7, fbsrggog, y14lw5b2, y047cuxp, 2arx86a6, ncbjncfx, 2q6qmex3, a3qf6i28, aahsvh09, 0c03fjk5, 3r8jbhhq, v1egyqt4, 2yka2luy, q55l1chd, p4ejdth1, rvygtpvb, tmp6yxlv, oijl3pjt, xpqfpizp, rwh56zhg, lt7qsxxh, 4zelg0m1, ...]","[uppeztta, rcb943w2, aahsvh09, 8rv94jeu, 0rrhgz26, 9sv03cqa, v1egyqt4, o29hjkca, s7z1l3a9, to2wxs8m]"
11094,i just had a terrifying realization: hiv attaches with cd4 &gt; the body absorbs and suppresses cd4 &gt; long term - no cd4 t cells sars-cov-2 attaches with ace2 &gt; the body absorbs and suppresses ace2 &gt; long term - out of control raas (ang ii),wku1sd9k,"[s86zb5up, f5g3mcee, geo7ac5i, 25aj8rj5, xwax7o13, b52pn8t9, tov6uq27, 65fwicjz, 2u9eenwu, k6cumncp, t2gxkxxd, jzosdlu7, tffpan0f, 2jdlavwj, wku3qrtg, qv31t2vh, fxwszm22, est5jx7g, far6giyb, 0r46eacc, 0r46eacc, izlg8zu6, pspko62f, xcs9podj, 4unn3fmu, 1pbo1qlb, vmmwtdia, yr6z0eki, 8r6hln9b, 0ojzssli, jyxrk9bz, ltet5qu6, rzahax88, zmxmo1q8, n1q1wr9s, yrmeweat, utc0qrax, mcw3ir3a, h82s5xst, nc2sh98g, 6sy80720, bn4rpjv0, 50dmclk4, n3upa0xn, llo64qbp, n2wqalcu, tqx25jad, lehzj4d8, a64w0a1y, thaje9nj, hdjlnot1, aaxhrecp, 446p4tbc, 1iago9rr, ky5env7t, 4dsq1dds, a5av0dq8, g9ao6ruo, 7jwqm0b2, k1w2wxnu, k4ttbbix, 6vqf2n5j, 8xf8u6bg, 5tpkcd5z, 0hqe4jb5, ndhhy1xt, ruw45n81, hyezouy8, 8rvhqd9b, jb455t9p, m4u4ulml, l3k9bb5m, 89gzlrd1, xi17qo6z, ikoyaj3b, f6sm0w9y, 2199ydle, c187k4yc, irk1gxeq, b22cioi2, 6vkab51a, 4pyx0xps, 9y9s7tn3, kde51mn0, itx70h27, tv1a1hwj, o4vvlmr4, waerqfzu, wpuujmjx, 2i7zd1t2, hgg33kwz, lz8b60ew, 58r5wcwd, erygg5u2,

In [ ]:
results = get_performance_mrr(df_query, 'cord_uid', 'bm25_cross_encoder_topk')
# Printed MRR@k results in the following format: {k: MRR@k}
print(results)

{1: 0.55, 5: 0.5916666666666666, 10: 0.5916666666666666}
